In [75]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import VGG16
from tensorflow.keras.layers import Dense, Flatten, Dropout
from tensorflow.keras.models import Model

In [76]:
IMG_SIZE = (224, 224)  # VGG16 expects 224x224 images
BATCH_SIZE = 32

In [77]:
train_dir = "./train"
test_dir = "./test"

In [78]:
train_datagen = ImageDataGenerator(rescale=1./255, horizontal_flip=True, vertical_flip=True)
test_datagen = ImageDataGenerator(rescale=1./255)

In [79]:
train_datagen = ImageDataGenerator(
    rescale=1./255,
    horizontal_flip=True,
    vertical_flip=True,
    rotation_range=30,
    zoom_range=0.2,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    fill_mode='nearest'
)

In [80]:
test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical'
)

Found 546 images belonging to 3 classes.


In [81]:
vgg16_base = VGG16(weights='imagenet', include_top=False, input_shape=IMG_SIZE + (3,))

In [87]:
for layer in vgg16_base.layers[:-4]:  # Unfreeze last 4 layers
    layer.trainable = True

In [88]:
x = vgg16_base.output
x = Flatten()(x)  # Flatten the feature maps
x = Dense(512, activation='relu')(x)
x = Dropout(0.5)(x)
x = Dense(3, activation='softmax')(x)

In [94]:
model = Model(inputs=vgg16_base.input, outputs=x)

SyntaxError: positional argument follows keyword argument (1958327189.py, line 1)

In [92]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [93]:
history = model.fit(
    train_generator,
    validation_data=test_generator,
    epochs=50,  # Train for more epochs
    callbacks=[tf.keras.callbacks.EarlyStopping(patience=10, restore_best_weights=True)]
)

Epoch 1/50
  2/138 ━━━━━━━━━━━━━━━━━━━━ 34:38 15s/step - accuracy: 0.4531 - loss: 1.5624

KeyboardInterrupt: 